<a href="https://colab.research.google.com/github/darisoy/sudoku/blob/main/sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np

In [ ]:
def print_sudoku(puzzle):
  for r, row in enumerate(puzzle):
    pr = ""
    for c, col in enumerate(row):
      pr += str(col)
      if c == 2 or c == 5:
        pr += "|"  
    print(pr)
    if r == 2 or r == 5:
      print("---|---|---")

In [ ]:
def box(puzzle, x, y):
  col = [6, 7, 8]
  if x < 3:
    col = [0, 1, 2]
  elif (x >= 3 and x < 6):
    col = [3, 4, 5]

  row = [6, 7, 8]
  if y < 3:
    row = [0, 1, 2]
  elif (y >= 3 and y < 6):
    row = [3, 4, 5]

  box = []
  for c in col:
    for r in row:
      box.append(puzzle[r][c])
  return box

def col(puzzle, y):
  c = []
  for row in puzzle:
    c.append(row[y])
  return c

def row(puzzle, x):
  return puzzle[x]

def valid_number(puzzle, n, x, y):
  return (n not in box(puzzle, x, y)) and (n not in col(puzzle, x)) and (n not in row(puzzle, y))

def pick_valid_number(puzzle, possible, x, y):
  for n in possible:
    if valid_number(puzzle, n, x, y):
      return n

In [ ]:
def generate_solution(verbose=False):
  s = [[0 for i in range(9)] for j in range(9)]
  possible = [[random.sample(range(1, 10), 9) for _ in range(9)] for _ in range(9)]
  x, y = 0, 0
  while x < 9 and x >= 0:
    while y < 9 and y >= 0 and x >= 0:
      num = pick_valid_number(s, possible[y][x], x, y)
      if verbose:
        print("(%d, %d) = %s" % (x, y, str(num)))
        print(possible[y][x])
      if num == None:
        s[y][x] = 0
        possible[y][x] = random.sample(range(1, 10), 9)
        y -= 1
        if y < 0:
          x -= 1
          y = 0
      else:
        s[y][x] = num
        if num in possible[y][x]:
          possible[y][x].remove(num)
        y += 1
      if verbose:
        print_sudoku(s)
    y = 0
    x += 1
  return s

In [ ]:
def unique(puzzle):
  s = [row[:] for row in puzzle] 
  return solutions(s, 0, 0, 0) == 1

def solutions(puzzle, x, y, count):
  if x == 9:
    x = 0
    y += 1
    if y == 9:
      return count + 1
  
  if puzzle[y][x] != 0:
    return solutions(puzzle, x+1, y, count)

  for n in range(1, 10):
    if count > 2:
      return count
    if valid_number(puzzle, n, x, y):
      puzzle[y][x] = n
      count = solutions(puzzle, x+1, y, count)
  
  puzzle[y][x] = 0
  return count

In [ ]:
def generate_puzzle(puzzle):
  p = [row[:] for row in puzzle]
  cells = [i for i in range(81)]
  while len(cells) > (81 - random.randint(35, 65)):
    i = random.choice(cells)
    x, y = int(i / 9), i % 9
    n = p[y][x]
    p[y][x] = 0
    if not unique(p):
      p[y][x] = n
    else:
      cells.remove(i)
  return p

In [ ]:
class Sudoku:
  def __init__(self):
    self.solution = generate_solution()
    self.puzzle = generate_puzzle(self.solution)

In [ ]:
def solve(puzzle):
  s = [row[:] for row in puzzle]
  to_solve = []
  for x in range(9):
    for y in range(9):
      if puzzle[y][x] == 0:
        to_solve.append((x, y))

  orig_possible = []
  for x, y in to_solve:
    new_possible = []
    for p in range(1, 10):
      if valid_number(puzzle, p, x, y):
        new_possible.append(p)
    orig_possible.append(new_possible)

  possible = [row[:] for row in orig_possible]
  i = 0
  while i < len(to_solve):
    x, y = to_solve[i]
    num = pick_valid_number(s, possible[i], x, y)
    if num is None:
      s[y][x] = 0
      possible[i] = orig_possible[i][:]
      i -= 1
    else:
      s[y][x] = num
      if num in possible[i]:
        possible[i].remove(num)
      i += 1
  return s

In [ ]:
b = Sudoku()
print("Solution:")
print_sudoku(b.solution)
print()
print("Puzzle:")
print_sudoku(b.puzzle)
print()
print("Solved result:")
print_sudoku(solve(b.puzzle))
print()
print("Solved is correct?")
print(np.array_equal(solve(b.puzzle), b.solution))

Solution:
528|967|413
947|531|862
361|248|579
---|---|---
674|325|981
832|719|654
195|684|327
---|---|---
219|853|746
453|176|298
786|492|135

Puzzle:
500|007|403
040|001|862
001|208|579
---|---|---
004|025|081
832|009|650
005|004|300
---|---|---
010|003|740
400|076|290
086|092|130

Solved result:
528|967|413
947|531|862
361|248|579
---|---|---
674|325|981
832|719|654
195|684|327
---|---|---
219|853|746
453|176|298
786|492|135

Solved is correct?
True
